In [97]:

import qualified Torch.NN.Recurrent.Cell.LSTM as LSTM
import qualified Torch.NN.Recurrent.Cell.GRU as GRU
import qualified Torch.Functional as F
import qualified Torch.Functional.Internal as FI
import Torch
import Torch.NN
import Torch.Autograd

In [98]:
x <- randIO' [16,7] 
y <- randIO' [16,1]
shape x

[16,7]

In [99]:
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE RecordWildCards #-}

import GHC.Generics (Generic)

data MLPSpec = MLPSpec
  { inputFeatures :: Int,
    hiddenFeatures0 :: Int,
    hiddenFeatures1 :: Int,
    outputFeatures :: Int
  }
  deriving (Show, Eq)

data MLP = MLP
  { l0 :: Linear,
    l1 :: Linear,
    l2 :: Linear
  }
  deriving (Generic, Show, Parameterized)

instance Randomizable MLPSpec MLP where
  sample MLPSpec {..} =
    MLP
      <$> sample (LinearSpec inputFeatures hiddenFeatures0)
      <*> sample (LinearSpec hiddenFeatures0 hiddenFeatures1)
      <*> sample (LinearSpec hiddenFeatures1 outputFeatures)

mlp :: MLP -> Tensor -> Tensor
mlp MLP {..} =

    linear l2 . relu . linear l1 . relu . linear l0

In [100]:
spec = MLPSpec 7 64 32 1
model <- sample spec

In [101]:
result = mlp model x
shape result

[16,1]

In [102]:
processBatch :: Optimizer o => MLP -> o -> Float -> (Tensor, Tensor) -> IO (MLP, Float)
processBatch model optimizer lr (input, label) = do
  let output = mlp model input
  let loss = FI.mse_loss output label 1  
  (newModel, _) <- runStep model optimizer loss (realToFrac lr)  
  pure (newModel, asValue loss)

In [103]:
x <- randIO' [16,7]
y <- randIO' [16,1]
optimizer = GD
lr = 0.001
batch = (x,y)

(model,loss) <- processBatch model optimizer lr batch

In [104]:

import Control.Monad (foldM)

processTrain :: Optimizer o => [(Tensor, Tensor)] -> MLP -> o -> Float -> IO (MLP, Float)
processTrain dataloader model optimizer lr = do
  foldM (\(m, _) (i, batch) -> do
            (newModel, loss) <- processBatch m optimizer lr batch
            putStrLn $ "Batch " ++ show i ++ " : loss : " ++ show loss
            pure (newModel, loss)
        ) (model, 0.0) (zip [1..] dataloader)


In [105]:
dataloader = [batch,batch,batch]
(model,loss) <- processTrain dataloader model optimizer lr
loss

Batch 1 : loss : 0.42876917
Batch 2 : loss : 0.42479417
Batch 3 : loss : 0.4208717

0.4208717